In [ ]:
# !/usr/bin/python3 -m pip install ipykernel -U --user --force-reinstall

In [ ]:
# !pip3 install pyvis
!pip3 install bokeh

In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import html
from urllib import parse
import requests
import networkx
from bokeh.io import output_notebook, show, save
output_notebook()
# from pyvis.network import Network

In [ ]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

In [4]:
from utils import *

In [ ]:
# with open('./soc-pokec-profiles.txt', 'r') as f:
#     profiles = f.readlines()
# with open('./soc-pokec-relationships.txt', 'r') as f:
#     relationships = f.readlines()

# max_node = 500

# relationships = relationships
# profiles = profiles[:max_node]
# print(len(profiles))

# with open('profiles_200_sample.txt', 'w') as f:
#     for l in profiles:
#         f.write(l)

# # with open('relationships_50_sample.txt', 'w') as f:
# #     for l in relationships:
# #         f.write(l)

# with open('profiles_200_sample.txt', 'r') as f:
#     profiles = f.readlines()
# print(len(profiles))
# def get_empty_user(profiles):

#     pp = [parse_row(p) for p in profiles]
#     all_node_id = []
#     for p in pp:
#         all_node_id.append(int(p[0]))

#     missing = []
#     for i in range(1, 501):
#         if i not in all_node_id:
#             missing.append(i)
#     return missing

# print(len(get_empty_user(profiles)))

In [ ]:
node_attributes = ['user_id', 'public', 'completion_percentage', 'gender', 'region', 'last_login', 'registration', 'AGE', 'body', 'I_am_working_in_field', 'spoken_languages', 'hobbies', 'I_most_enjoy_good_food', 'pets', 'body_type', 'my_eyesight', 'eye_color', 'hair_color', 'hair_type', 'completed_level_of_education', 'favourite_color', 'relation_to_smoking', 'relation_to_alcohol', 'sign_in_zodiac', 'on_pokec_i_am_looking_for', 'love_is_for_me', 'relation_to_casual_sex', 'my_partner_should_be', 'marital_status', 'children', 'relation_to_children', 'I_like_movies', 'I_like_watching_movie', 'I_like_music', 'I_mostly_like_listening_to_music', 'the_idea_of_good_evening', 'I_like_specialties_from_kitchen', 'fun', 'I_am_going_to_concerts', 'my_active_sports', 'my_passive_sports', 'profession', 'I_like_books', 'life_style', 'music', 'cars', 'politics', 'relationships', 'art_culture', 'hobbies_interests', 'science_technologies', 'computers_internet', 'education', 'sport', 'movies', 'travelling', 'health', 'companies_brands', 'more']

In [ ]:
vis_attributes = ['user_id', 'public', 'completion_percentage', 'gender', 'region', 'last_login', 'registration', 'AGE', 'body', 'I_am_working_in_field', 'spoken_languages', 'hobbies']

In [ ]:
def load_from_csv(path):
    df = pd.read_csv(path)
    return df

def row_to_dict(row, attr):
    data = {}
    for a in attr:
        data[a] = row[a]
    return data

In [ ]:
def get_graph_edge(max_node=20, empty_user=[]):
    
    edge_df = pd.read_csv('./toy_example_edge_50.csv')
    G = networkx.from_pandas_edgelist(edge_df, 'source', 'target')
    print('Edge Amount of this graph:', len(edge_df))
    return G

In [ ]:
def create_network(profiles, attr, max_node, empty_user):
    
    net = get_graph_edge(max_node, empty_user)
    # profiles : dataframe

    all_attrs = {}
    for idx, p in profiles.iterrows():
        node_attr = row_to_dict(p, attr)
        node_id = int(p['user_id'])
        if node_id <= max_node:
            all_attrs[node_id] = node_attr
    networkx.set_node_attributes(net, all_attrs)
    return net

In [ ]:
def load_missing():
    with open('./missing_users.txt') as f:
        lines = f.readlines()
    lst = [int(l.resplace('\n', '')) for l in lines]
    return lst

missing = load_missing()

In [ ]:
df = pd.read_csv('./toy_example_500.csv')
net = create_network(df, node_attributes, 50, missing)

In [ ]:
print(net.nodes) # node id
net.nodes[31]

In [ ]:
title = 'Testing Network'

HOVER_TOOLTIPS = [(p, "@"+p) for p in vis_attributes]
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

network_graph = from_networkx(net, networkx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
plot.renderers.append(network_graph)
show(plot)